In [3]:
############################################################################################################

# Peter Bui
# CSCI-580: 3-D Graphics and Rendering
# Homework 4

############################################################################################################


from PIL import Image
import json
import math
import random
import numpy as np

im = Image.new('RGB', [512,512], 0x000000)
width, height = im.size

# loading JSON data
with open('./teapot.json') as json_file:
    data = json.load(json_file)
with open('./scene.json') as scene_file: 
    data2 = json.load(scene_file)
    
# fill in background w/ color from sample
for y in range(height):
    for x in range(width):
        im.putpixel((x,y), (0,0,0))

# initialize z-buffer to positive infinity
zBuffer = []
for a in range(height):
    bList = []
    for b in range(width):
        bList.append(float('inf'))
    zBuffer.append(bList)

    
# ----------------------------------------------------------------------------------------------------------
# Clips coordinates to ensure they are within the valid range.
# Inputs:
#   - v: A tuple representing values to be clipped.
# Output:
#   - Returns the clipped tuple.
# Usage:
#   - Called in scanTriangle() to ensure vertex coordinates are within valid image boundaries.
def clipCoords(v):
    # 1. Convert the tuple to a list to modify -- return it again as a tuple.
    v = list(v)
    v[0] = max(0, min(512, v[0]))
    v[1] = max(0, min(512, v[1]))
    v[2] = max(0, min(512, v[2]))
    return tuple(v)


# ----------------------------------------------------------------------------------------------------------
# Calculate the dot product between 2 vectors. 3-D vectors ONLY!
# Inputs:
#   - A: python list
#   - B: python list
# Output:
#   - C: value of dot product
def dot(A, B):
    return abs(A[0]*B[0] + A[1]*B[1] + A[2]*B[2])


# ----------------------------------------------------------------------------------------------------------
# Calculate the cross product between 2 vectors. 3-D vectors ONLY!
# Inputs:
#   - A: python list
#   - B: python list
# Output:
#   - C: list that represents cross product of A and B
def cross(A, B):
    cross_product = []
    cross_product.append((A[1]*B[2] - A[2]*B[1]))
    cross_product.append((A[2]*B[0] - A[0]*B[2]))
    cross_product.append((A[0]*B[1] - A[1]*B[0]))
    return cross_product


# ----------------------------------------------------------------------------------------------------------
# Calculate the length (magnitude) of a vector. Works for all lengths.
# Inputs:
#   - V: python list representing a vector
# Output:
#   - value representing magnitude length of V
def magnitude(V):
    squared_sum = 0
    for x in V:
        squared_sum += x ** 2
    return math.sqrt(squared_sum)


# ----------------------------------------------------------------------------------------------------------
# Extracts unit vector from vector by dividing each element by its magnitude. Works for all lengths.    
# Inputs:
#   - V: python list representing a vector
# Output:
#   - the unit vector of V
def toUnit(V):
    l = magnitude(V)
    if l != 0:
        unit_vector = []
        for x in V:
            unit_vector.append(x / l)
        return unit_vector
    else:
        return [0] * len(V)  # Return zero vector if magnitude is zero

    
# ----------------------------------------------------------------------------------------------------------
# Calculate the color of a triangle based on its normal vector and apply a "tint".
# Inputs:
#   - normal: A tuple representing the normal vector of the triangle.
# Output:
#   - Returns a list containing three values representing the RGB color.
# Usage:
#   - Called for each triangle in the main processing loop to determine its color.
#   - Ensures that the color values are within the valid range (0 to 255).
def computeTriangleColor(normal):
    
    # 1. Compute the dot product between our triangle's FIRST normal and an assumed light vector
    dotp = dot(normal, [0.707, 0.5, 0.5])
    
    # 2. Clamp the dot product to 0..1, which would give you a gray value for the entire tri
    if (dotp < 0.0):
        dotp = -dotp
    elif (dotp > 1.0):
        dotp = 1.0
    
    # 3. Convert it to a purplish hue [for no good reason!]
    triangleR = float(0.95)*dotp
    triangleG = float(0.65)*dotp
    triangleB = float(0.88)*dotp
    return [triangleR, triangleG, triangleB]


# ----------------------------------------------------------------------------------------------------------
# Constructs a camera matrix representing the transformation from world space to camera space.
# Inputs:
#   - r: A tuple representing the camera position.
#   - P: A tuple representing the point to look at.
# Output:
#   - Returns a 4x4 camera matrix.
# Note:
#   - y-axis is considered the "up" direction.
#   - [0, 1, 0] represents a unit vector pointing straight up along the positive y-axis
def createCamMatrix(r, P):
    # n: unit vector pointing in the opposite direction of the camera's viewing direction
    n = toUnit([r[0]-P[0], r[1]-P[1], r[2]-P[2]])
    # u: unit vector pointing to the right in the camera's coordinate system
    u = toUnit(cross([0, 1, 0], n))
    # v: unit vector pointing upwards in the camera's coordinate system
    v = toUnit(cross(n, u))
    
    cameraMatrix = [[u[0], u[1], u[2], -dot(r, u)],
                    [v[0], v[1], v[2], -dot(r, v)],
                    [n[0], n[1], n[2], -dot(r, n)],
                    [   0,    0,    0,         1]]
    return cameraMatrix


# ----------------------------------------------------------------------------------------------------------
# Converts coordinates from world space to camera space using the camera matrix
# Inputs:
#   - coordinate: A tuple representing the 3D coordinate in world space.
# Output:
#   - Returns a tuple representing the 3D coordinate in camera space.
def worldToCam(coordinate, toCam):
    result1 = np.matmul(toCam, coordinate)
    return perspectiveProjection(result1)


# ----------------------------------------------------------------------------------------------------------
# Performs perspective projection on the transformed coordinates.
# Inputs:
#   - result1: A tuple representing the transformed coordinates in camera space.
# Output:
#   - Returns a tuple representing the projected coordinates.
def perspectiveProjection(result1):
    
    near, far, left, right, top, bottom = data2['scene']['camera']['bounds']

    # perspective projection matrix
    perspectiveProj = np.array([[2*near/(right-left), 0, (right+left)/(right-left), 0],
                                [0, 2*near/(top-bottom), (top+bottom)/(top-bottom), 0],
                                [0, 0, -(far+near)/(far-near), -(2*far*near)/(far-near)],
                                [0, 0, -1, 0]])

    result2 = np.matmul(perspectiveProj, result1)
    return toNDC(result2)


# ----------------------------------------------------------------------------------------------------------
# Converts projected coordinates to Normalized Device Coordinates (NDC).
# Inputs:
#   - result2: A tuple representing the projected coordinates.
# Output:
#   - Returns a tuple representing the NDC coordinates.
def toNDC(result2):
    result3 = []
    for p in range(3):
        if result2[3] != 0:
            result3.append(result2[p]/result2[3])
        else:
            result3.append(result2[p])
    return toRasterSpace(result3)


# ----------------------------------------------------------------------------------------------------------
# Maps NDC to raster space (2D image space).
# Inputs:
#   - result3: A tuple representing the NDC coordinates.
# Output:
#   - Returns a tuple representing the coordinates mapped to the 2D image space.
def toRasterSpace(result3):
    xND = result3[0] 
    yND = result3[1]
    zND = result3[2]
    xR = (xND+1)*((width-1)/2)
    yR = (1-yND)*((height-1)/2)
    zR = (zND+1)*((512-1)/2)
    return (xR, yR, zR)


# ----------------------------------------------------------------------------------------------------------
# Rotate a vector around the y-axis by a specified angle in degrees.
# Inputs:
#   - R: The angle of rotation in degrees.
#   - V: A numpy array representing the 3D vector to be rotated.
# Output:
#   - Returns a numpy array representing the rotated vector.
def rotateY(R,V):
    theta = -math.radians(R)
    rMatrix = np.array([[ np.cos(theta), 0, np.sin(theta), 0],
                        [             0, 1,             0, 0],
                        [-np.sin(theta), 0, np.cos(theta), 0],
                        [             0, 0,             0, 1]]) 
    return np.matmul(rMatrix, V)


# ----------------------------------------------------------------------------------------------------------
# Scale a vector by the specified scaling factors along each axis.
# sx, sy, sz = S[0:2]
# Inputs:
#   - S: A list or numpy array containing the scaling factors along the x, y, and z axes.
#   - V: A numpy array representing the 3D vector to be scaled.
# Output:
#   - Returns a numpy array representing the scaled vector.
def scale(S, V):
    sMatrix = np.array([[S[0],0,0,0],[0,S[1],0,0],[0,0,S[2],0],[0,0,0,1]])
    return np.matmul(sMatrix, V)


# ----------------------------------------------------------------------------------------------------------
# Translate a vector by the specified translation amounts along each axis.
# tx, ty, tz = T[0:2]
# Inputs:
#   - T: A list or numpy array containing the translation amounts along the x, y, and z axes.
#   - V: A numpy array representing the 3D vector to be translated.
# Output:
#   - Returns a numpy array representing the translated vector.
def translate(T, V):
    tMatrix = np.array([[1,0,0,T[0]],[0,1,0,T[1]],[0,0,1,-T[2]],[0,0,0,1]])
    return np.matmul(tMatrix, V)


# ----------------------------------------------------------------------------------------------------------
# Normalize a given vector to unit length.
# vx, vy, vz = V[0:2]
# Inputs:
#   - nVector: A numpy array representing the vector to be normalized.
# Output:
#   - Returns a numpy array representing the normalized vector.
def normalize(V): 
    n = magnitude([V[0],V[1],V[2]]) 
    V[0] = V[0]/n 
    V[1] = V[1]/n 
    V[2] = V[2]/n
    for w in range(3):
        if math.isnan(float(V[w])):
            V[w] = 0
    return V


# ----------------------------------------------------------------------------------------------------------
# Compute the ADS (Ambient, Diffuse, Specular) lighting model for a given surface point.
# Inputs:
#   - Cs: A list containing three values representing the RGB color of the surface.
#   - normal: A numpy array representing the normal vector at the surface point.
#   - Kd: The diffuse reflection coefficient.
#   - Ka: The ambient reflection coefficient.
#   - Ks: The specular reflection coefficient.
#   - Ie: A numpy array representing the color and intensity of the directional light source.
#   - Ia: A numpy array representing the color and intensity of the ambient light source.
#   - n: The specular exponent for controlling the specular highlight.
#   - lights: A list containing information about all light sources.
#   - dirLightPath: A numpy array representing the direction of the directional light source.
# Output:
#   - Returns a list containing three values representing the RGB color after applying the ADS model.
def ADS(Cs, normal, Kd, Ka, Ks, Ie, Ia, n, lights, dirLightPath):
    N = normalize(np.array(normal)) # normalize surface normal vector
    L = normalize(dirLightPath)     # normalize light ray direction vector
    E = [0,1,1]                     # eye ray direction vector
    
    # if both negative, flip normal
    if (dot(N,L)<0 and dot(N,E)<0):
        N = -N
        
    # normalized reflected ray direction vector
    NL = dot(N,L)
    R = normalize(np.array([2*NL*N[0]-L[0], 2*NL*N[1]-L[1], 2*NL*N[2]-L[2]]))
    
    # bounding RE dot product between 0 and 1
    RE = dot(R,E)
    if (RE > 1): RE = 1;
    if (RE < 0): RE = 0;
        
    sumSpec = Ks*Ie*(RE**n)
    sumDiffuse = Kd*Ie*NL
    ADScolor = sumSpec+sumDiffuse+(Ka*Ia)
    Cs = Cs*ADScolor
    
    return Cs


# ----------------------------------------------------------------------------------------------------------
# Perform Gouraud shading to compute the color at a vertex and interpolate colors across a triangle.
# Inputs:
#   - vertexMatrix: A 3x3 numpy array representing the vertices of the triangle.
#   - alpha, beta, gamma: Barycentric coordinates for interpolation.
#   - Cs: A list containing three values representing the RGB color of the triangle.
#   - normal: A numpy array representing the normal vector at the vertex.
#   - Kd, Ka, Ks: Diffuse, ambient, and specular reflection coefficients.
#   - dirLight: A numpy array representing the color and intensity of the directional light source.
#   - ambLight: A numpy array representing the color and intensity of the ambient light source.
#   - n: The specular exponent for controlling the specular highlight.
#   - lights: A list containing information about all light sources.
#   - dirLightPath: A numpy array representing the direction of the directional light source.
# Output:
#   - Returns a list containing three values representing the RGB color at the vertex.
def gouraud(vertexMatrix, alpha, beta, gamma, Cs, normal, Kd, Ka, Ks, dirLight, ambLight, n, lights, dirLightPath):
    ADSresult = []
    # ADS calculation for vertices
    result1 = ADS(Cs, [normal[0][0],normal[0][1],normal[0][2]], Kd, Ka, Ks, dirLight, ambLight, n, lights, dirLightPath)
    result2 = ADS(Cs, [normal[1][0],normal[1][1],normal[1][2]], Kd, Ka, Ks, dirLight, ambLight, n, lights, dirLightPath)
    result3 = ADS(Cs, [normal[2][0],normal[2][1],normal[2][2]], Kd, Ka, Ks, dirLight, ambLight, n, lights, dirLightPath)
    # interpolate RGB and append to ADSresult
    ADSresult.append((result1[0]*alpha)+(result2[0]*beta)+(result3[0]*gamma))
    ADSresult.append((result1[1]*alpha)+(result2[1]*beta)+(result3[1]*gamma))
    ADSresult.append((result1[2]*alpha)+(result2[2]*beta)+(result3[2]*gamma))
    return ADSresult


# ----------------------------------------------------------------------------------------------------------
# Perform Phong shading to interpolate normals and compute the color at each pixel of a triangle.
# Inputs:
#   - vertexMatrix: A 3x3 numpy array representing the vertices of the triangle.
#   - alpha, beta, gamma: Barycentric coordinates for interpolation.
#   - Cs: A list containing three values representing the RGB color of the triangle.
#   - normal: A numpy array representing the normal vector at the vertex.
#   - Kd, Ka, Ks: Diffuse, ambient, and specular reflection coefficients.
#   - dirLight: A numpy array representing the color and intensity of the directional light source.
#   - ambLight: A numpy array representing the color and intensity of the ambient light source.
#   - n: The specular exponent for controlling the specular highlight.
#   - lights: A list containing information about all light sources.
#   - dirLightPath: A numpy array representing the direction of the directional light source.
# Output:
#   - Returns a list containing three values representing the RGB color at each pixel of the triangle.
def phong(vertexMatrix, alpha, beta, gamma, Cs, normal, Kd, Ka, Ks, dirLight, ambLight, n, lights, dirLightPath):
    N0 = (normal[0][0] * alpha) + (normal[1][0] * beta) + (normal[2][0] * gamma)
    N1 = (normal[0][1] * alpha) + (normal[1][1] * beta) + (normal[2][1] * gamma)
    N2 = (normal[0][2] * alpha) + (normal[1][2] * beta) + (normal[2][2] * gamma)
    return ADS(Cs, [N0,N1,N2], Kd, Ka, Ks, dirLight, ambLight, n, lights, dirLightPath)


# ----------------------------------------------------------------------------------------------------------
# Perform scan conversion for a single triangle, filling pixels inside its bounding box.
# Check if pixels lay within the triangle using barycentric coordinates.
# Inputs:
#   - GouraudOrPhong: takes 'gouraud', 'Gouraud', 'Phong', or 'phong' to indicate shading technique.
#   - v0, v1, v2: Tuples representing (x, y, z) coordinates of the triangle vertices.
#   - normalMatrix: A 3x4 numpy array representing the normals of the triangle vertices.
#   - Cs: A list containing three values representing the RGB color of the triangle.
#   - Kd: The diffuse reflection coefficient.
#   - Ka: The ambient reflection coefficient.
#   - Ks: The specular reflection coefficient.
#   - dirLight: A numpy array representing the color and intensity of the directional light source.
#   - ambLight: A numpy array representing the color and intensity of the ambient light source.
#   - n: The specular exponent for controlling the specular highlight.
#   - lights: A list containing information about all light sources.
#   - dirLightPath: A numpy array representing the direction of the directional light source.
# Usage:
#   - Called in a loop for each triangle in the main processing loop.
#   - Computes lighting and shading effects using the Gouraud or Phong shading model based on the value of 'gouraudNotPhong'.
def scanTriangle(GouraudOrPhong, v0, v1, v2, normalMatrix, Cs, Kd, Ka, Ks, dirLight, ambLight, n, lights, dirLightPath):
    
    # 1. Calculate xmax, xmin, ymax, ymin
    
    normalVector = normalMatrix
    v0_clipped = clipCoords(v0)
    v1_clipped = clipCoords(v1)
    v2_clipped = clipCoords(v2)
    xVals = [v0_clipped[0], v1_clipped[0], v2_clipped[0]]
    yVals = [v0_clipped[1], v1_clipped[1], v2_clipped[1]]
    xmin = int(math.floor(min(xVals)))
    xmax = int(math.ceil(max(xVals)))
    ymin = int(math.floor(min(yVals)))
    ymax = int(math.ceil(max(yVals)))
    
    # 2. Define method for lines f01, f12, f20
    
    x0, y0, z0 = v0
    x1, y1, z1 = v1
    x2, y2, z2 = v2
    def f01(x,y):
        return (y0-y1)*x + (x1-x0)*y + x0*y1-x1*y0
    def f12(x,y):
        return (y1-y2)*x + (x2-x1)*y + x1*y2-x2*y1
    def f20(x,y): 
        return (y2-y0)*x + (x0-x2)*y + x2*y0-x0*y2
    
    # 3. Update the zBuffer and place pixel on image (if needed)
    
    for y in range(ymin, ymax):
        for x in range(xmin, xmax):
            
            # checking alpha, beta, gamma
            if (f12(x0,y0) == 0 or f20(x1,y1) == 0 or f01(x2,y2) == 0):
                continue
            alpha = f12(x,y) / f12(x0,y0)
            beta  = f20(x,y) / f20(x1,y1)
            gamma = f01(x,y) / f01(x2,y2)
            
            if ((alpha >= 0) and (beta >= 0) and (gamma >= 0)):
                zPixel = alpha*z0 + beta*z1 + gamma*z2
                
                if zPixel < zBuffer[x][y]:
                    vertexMatrix = [v0_clipped, v1_clipped, v2_clipped]
                    
                    if GouraudOrPhong == 'gouraud' or GouraudOrPhong == 'Gouraud':
                        finalColor = gouraud(vertexMatrix, alpha, beta, gamma, Cs, normalVector, Kd, Ka, Ks, dirLight, ambLight, n, lights, dirLightPath)
                    elif GouraudOrPhong == 'phong' or GouraudOrPhong == 'Phong':
                        finalColor = phong(vertexMatrix, alpha, beta, gamma, Cs, normalVector, Kd, Ka, Ks, dirLight, ambLight, n, lights, dirLightPath)
                    
                    r = max(0, min(255, int(math.floor(finalColor[0] * 256.0))))
                    g = max(0, min(255, int(math.floor(finalColor[1] * 256.0))))
                    b = max(0, min(255, int(math.floor(finalColor[2] * 256.0))))
                    
                    zPixel = alpha*z0 + beta*z1 + gamma*z2
                    im.putpixel((x,y),(r,g,b))
                    zBuffer[x][y] = zPixel


############################################################################################################
###########################################   IMAGE RENDERING  #############################################
############################################################################################################

shapes = data2['scene']['shapes']                       # no adjustments
lights = data2['scene']['lights']                       # no adjustments

ambientColor = lights[0]['color']                       # no adjustments
directionalColor = lights[1]['color']                   # no adjustments

Ia = np.array(data2['scene']['lights'][0]['intensity']) # no adjustments
Ie = np.array(data2['scene']['lights'][1]['intensity']) # no adjustments

lightTo = lights[1]['to']                               # no adjustments
cameraTo = data2['scene']['camera']['to']               # no adjustments
cameraRes = data2['scene']['camera']['resolution']      # no adjustments

lightFrom = lights[1]['from']                           # adjustments
lightFrom[0] = -lightFrom[0]
lightFrom[1] = -lightFrom[1]
lightFrom[2] = lightFrom[2]

cameraFrom = data2['scene']['camera']['from']           # adjustments
cameraFrom[0] = cameraFrom[0]
cameraFrom[1] = cameraFrom[1]
cameraFrom[2] = -cameraFrom[2]


for q in shapes:
    
    Cs = q['material']['Cs']
    Ka = q['material']['Ka']
    Kd = q['material']['Kd']
    Ks = q['material']['Ks']
    n  = q['material']['n']
    Ry = q['transforms'][0]['Ry']
    ScaleMatrix = q['transforms'][1]['S']
    T = q['transforms'][2]['T']
    
    # Object -> World
    for q in range(len(data['data'])):
        
        # tuples representing the coordinates of the vertices of a triangle
        Coordinate1 = data['data'][q]['v0']['v']
        Coordinate2 = data['data'][q]['v1']['v']
        Coordinate3 = data['data'][q]['v2']['v']
        Normal1 = data['data'][q]['v0']['n']
        Normal2 = data['data'][q]['v1']['n']
        Normal3 = data['data'][q]['v2']['n']
        
        # converting to numpy array
        Coordinate1 = np.array([Coordinate1[0],Coordinate1[1],Coordinate1[2],1])
        Coordinate2 = np.array([Coordinate2[0],Coordinate2[1],Coordinate2[2],1])
        Coordinate3 = np.array([Coordinate3[0],Coordinate3[1],Coordinate3[2],1])  
        Normal1 = np.array([Normal1[0],Normal1[1],Normal1[2],1])
        Normal2 = np.array([Normal2[0],Normal2[1],Normal2[2],1])
        Normal3 = np.array([Normal3[0],Normal3[1],Normal3[2],1])
        
        # teapot.json*rotate 
        Coordinate1 = rotateY(Ry,Coordinate1)
        Coordinate2 = rotateY(Ry,Coordinate2)
        Coordinate3 = rotateY(Ry,Coordinate3)
        Normal1 = rotateY(Ry,Normal1)
        Normal2 = rotateY(Ry,Normal2)
        Normal3 = rotateY(Ry,Normal3)
        
        # scale(S,V)
        Coordinate1 = scale(ScaleMatrix,Coordinate1)
        Coordinate2 = scale(ScaleMatrix,Coordinate2)
        Coordinate3 = scale(ScaleMatrix,Coordinate3)
        
        # Inverse Normal Matrix
        inverseS = np.array([1/ScaleMatrix[0], 1/ScaleMatrix[1], 1/ScaleMatrix[2]])
        Normal1 = scale(inverseS,Normal1)
        Normal2 = scale(inverseS,Normal2)
        Normal3 = scale(inverseS,Normal3)
        
        # translate(T,V)
        Coordinate1 = translate(T,Coordinate1)
        Coordinate2 = translate(T,Coordinate2)
        Coordinate3 = translate(T,Coordinate3)
        
        # World -> Camera -> NDC
        toCam = createCamMatrix(cameraFrom, cameraTo)
        newC0 = worldToCam(Coordinate1, toCam)
        newC1 = worldToCam(Coordinate2, toCam)
        newC2 = worldToCam(Coordinate3, toCam)
        
        vertMatrix = np.array([newC0, newC1, newC2])
        normalMatrix = np.array([Normal1, Normal2, Normal3])
        
        # multiply light by camera matrix
        ambLight = np.array(ambientColor*Ia)
        dirLight = np.array(directionalColor*Ie)
        dirLightPath = (np.array(lightTo)-np.array(lightFrom))
        dirLightPath = [dirLightPath[0], dirLightPath[1], dirLightPath[2], 1]
        
        # SCAN TRIANGLE:
        # First input takes
        # 'gouraud' or 'Gouraud' for gouraud shading.
        # 'phong' or 'Phong' for phong shading.
        # other inputs will result in error.
        scanTriangle('phong', vertMatrix[0], vertMatrix[1], vertMatrix[2], normalMatrix,
                     Cs, Kd, Ka, Ks, dirLight, ambLight, n, lights, dirLightPath)

im = im.save("output.ppm")
print("Image output successfully.")

Image output successfully.
